In [1]:
import torch

torch.cuda.is_available()

True

## Simple Pytorch AutoGrad

![Auto Grad](/home/znyd/hacking/meow-former/images/auto_grad.png)

In [3]:
import torch.nn.functional as F
from torch.autograd import grad


y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)


z = x1 * w1 + b
a = F.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_w1 = grad(loss, w1, retain_graph=True)
grad_b = grad(loss, b, retain_graph=True)

# print(grad_w1)
# print(grad_b)

loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


## A Basic MLP

In [4]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(20, num_outputs),
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [11]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [12]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [13]:
X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.2422,  0.1378,  0.1153]], grad_fn=<AddmmBackward0>)


In [14]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.2569, 0.3757, 0.3673]])


## Basic Data Loader

In [15]:
X_train = torch.tensor([
[-1.2, 3.1],
[-0.9, 2.9],
[-0.5, 2.6],
[2.3, -1.1],
[2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])
X_test = torch.tensor([
[-0.8, 2.8],
[2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [16]:
from torch.utils.data import Dataset

class DummyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.lables = y
    def __getitem__(self, index):
        single_x = self.features[index]
        single_y = self.lables[index]
        return single_x, single_y
    def __len__(self):
        return self.lables.shape[0]
    

In [18]:
train_ds = DummyDataset(X_train, y_train)
test_ds = DummyDataset(X_test, y_test)

In [21]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [22]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-0.9000,  2.9000],
        [ 2.3000, -1.1000]]) tensor([0, 1])
Batch 2: tensor([[-0.5000,  2.6000],
        [-1.2000,  3.1000]]) tensor([0, 0])


## TODO: Multi-GPU training